# Import python packages

In [1]:
from pathlib import Path
import pymbe.api as pm

# Load the SysML v2 model from JSON

In [2]:
json_file = Path(Path.cwd()) / "example_data" / "TrashCollection.json"

boat_data = pm.Model.load_from_post_file(json_file)
boat_data

c:\git\pymbe\src\pymbe\label.py:115: UserWarning: Cannot process Expression elements yet!
  warn(f"Cannot process {expression._metatype} elements yet!")


<SysML v2 Model (C:\git\pymbe\dev_docs\data_loading\example_data\TrashCollection.json)>

## List and explore SysML packages in the model

In [3]:
packages = [ele for ele in boat_data.elements.values() if ele._metatype == 'Package']
packages

[Trash Collections «Package»]

In [4]:
analyses = \
    [ele for ele in packages[0].throughOwningMembership if ele._metatype == 'AnalysisCaseDefinition']
analyses

[Trash Collection Efficiency Analysis «AnalysisCaseDefinition»]

In [5]:
analyses[0].throughSubjectMembership[0]

Trash Removal Fleet: Trash Removal Fleet «ReferenceUsage»

In [6]:
fleet = analyses[0].throughSubjectMembership[0].throughFeatureTyping[0]
fleet

Trash Removal Fleet «PartDefinition»

In [7]:
generic_trash_removal_ship = fleet.throughFeatureMembership[0].throughFeatureTyping[0]

### Find trash capture ship alternatives

In [8]:
generic_trash_removal_ship.reverseSubclassification

[Water Surface Trash Skimmer «PartDefinition»,
 Net-Based Skimmer «PartDefinition»,
 Trash Processor Mothership «PartDefinition»]

In [9]:
surface_skimmer, net_skimmer, _ = generic_trash_removal_ship.reverseSubclassification
net_skimmer

Net-Based Skimmer «PartDefinition»

## Find the necessary parameters

### Find parameters for Surface Skimmer Concept

In [10]:
surface_skimmer.throughFeatureMembership

[Skim Trash: Skim Trash Def «PerformActionUsage»,
 Inlet Diameter: LengthValue «AttributeUsage»,
 Inlet Height: LengthValue «AttributeUsage»]

In [11]:
_, surface_skimmer_inlet_diameter, _ = surface_skimmer.throughFeatureMembership

# get the boat speed from the generic definition
surface_skimmer_speed_kts = surface_skimmer.throughSubclassification[0].throughFeatureMembership[0].throughFeatureValue[0].value

### Find parameters for Net Skimmer Concept

In [12]:
net_skimmer.throughFeatureMembership

[Net Width: LengthValue «AttributeUsage»,
 Time to Deploy: LengthValue «AttributeUsage»,
 Time to Gather: LengthValue «AttributeUsage»,
 boat speed: SpeedValue «AttributeUsage»]

In [13]:
net_skimmer_width, net_skimmer_time_to_deploy, _, net_skimmer_speed = net_skimmer.throughFeatureMembership
net_skimmer_time_to_deploy.throughFeatureValue[0].value

2.5

# Define a computable model

In [14]:
from math import ceil, sqrt

HRS_TO_YEARS = 1 / (24 * 365.25)


def time_to_skim_hrs(
    boat_speed_kts: float,
    boat_capture_width_m: float,
    boat_setup_time_hr: float,
    boat_garbage_capacity_m3: float,
    garbage_volume_density_m: float,  # volume of garbage per surface area (m3/m2 = m) to depth of capture
    area_to_skim_km2: float,
) -> float:
    """An overly simplistic model for how a trash skimming boat would clean up an area."""
    SEC_TO_HR = 1 / 3_600
    KM2_to_M2 = 1_000_000
    KTS_TO_MPS = 0.514444  # knots to meter/sec

    boat_speed_mps = KTS_TO_MPS * boat_speed_kts
    area_to_skim_m2 = KM2_to_M2 * area_to_skim_km2

    amount_of_trash_to_skim_m3 = area_to_skim_m2 * garbage_volume_density_m
    number_of_trips = ceil(amount_of_trash_to_skim_m3 / boat_garbage_capacity_m3)

    if number_of_trips == 1:
        time_to_skim_area_hr = SEC_TO_HR * area_to_skim_m2 / (boat_speed_mps * boat_capture_width_m)
        return boat_setup_time_hr + time_to_skim_area_hr

    print(f"Will need to do {number_of_trips:,d} trips to collect all the garbage")

    time_to_collect_to_capacity_hr = boat_garbage_capacity_m3 / (garbage_volume_density_m * boat_capture_width_m * boat_speed_mps) * SEC_TO_HR
    time_to_travel_to_mothership_hr = SEC_TO_HR * sqrt(KM2_to_M2 * area_to_skim_km2) / boat_speed_mps

    return number_of_trips * (boat_setup_time_hr + time_to_collect_to_capacity_hr + time_to_travel_to_mothership_hr)

# Define a Scenario

In [15]:
GARBAGE_VOLUME_DENSITY = 0.00001  # the volume of garbage per surface area (m3/m2)
AREA_TO_SKIM = 20  # km2

# Analyze Alternatives

## Simple skimmer

In [16]:
surface_skimmer_time_to_complete_hr = time_to_skim_hrs(
    boat_speed_kts=surface_skimmer_speed_kts,
    boat_capture_width_m=surface_skimmer_inlet_diameter.throughFeatureValue[0].value,
    boat_setup_time_hr=0.5,
    boat_garbage_capacity_m3=500,
    garbage_volume_density_m=GARBAGE_VOLUME_DENSITY,  
    area_to_skim_km2=AREA_TO_SKIM,
)
print(f"It will take the boat {int(surface_skimmer_time_to_complete_hr):,d} hrs to complete the task")

It will take the boat 270 hrs to complete the task


## Net-capture skimmer (using SysML2 model data)

In [17]:
net_skimmer_time_to_complete_hr = time_to_skim_hrs(
    boat_speed_kts=net_skimmer_speed.throughFeatureValue[0].value,
    boat_capture_width_m=net_skimmer_width.throughFeatureValue[0].value,
    boat_setup_time_hr=net_skimmer_time_to_deploy.throughFeatureValue[0].value,
    boat_garbage_capacity_m3=500,
    garbage_volume_density_m=GARBAGE_VOLUME_DENSITY,
    area_to_skim_km2=AREA_TO_SKIM,
)
print(f"It will take the boat {int(surface_skimmer_time_to_complete_hr):,d} hrs to complete the task")

It will take the boat 270 hrs to complete the task


## Combine analyses

In [18]:
print(f"Would need {surface_skimmer_time_to_complete_hr/net_skimmer_time_to_complete_hr:.2f} surface skimmer boats to be the equivalent of the net skimmer concept")

Would need 3.63 surface skimmer boats to be the equivalent of the net skimmer concept
